In [1]:
import pandas as pd
from prep_train_pipeline import prep_train_pipeline
from helper_roberta import tokenize, create_model
from transformers import RobertaTokenizerFast, TFRobertaModel
import tensorflow as tf

2024-03-26 11:40:24.224722: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 11:40:24.248612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 11:40:24.248634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 11:40:24.249212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 11:40:24.253097: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070, compute capability 8.9


2024-03-26 11:40:25.287461: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 11:40:25.308555: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 11:40:25.308685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# check gpu
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# read csv
df_512_token = pd.read_csv("text_512_tokens.csv")
df_512_token = df_512_token.drop(columns=["Unnamed: 0"])
df_512_token.head()

,text,target
0,introduction hi im a scientist at nasa and im ...,1.0
1,la mona lisa was created by one of the best dr...,0.0
2,have you ever heard of the seagoing cowboys pr...,0.0
3,online schooling has become increasingly popul...,1.0
4,many people and other scientists believe that ...,0.0


In [6]:
import random 

for i in range(20):
    random_idx = random.choice(df_512_token.index)
    values = df_512_token.loc[random_idx, ["text", "target"]]
    print(f"Target: {values[1]}, Text: {values[0]}\n\n")

Text: the automobile industry has been an essential part of american society for over a century however recent trends in american behavior suggest a cultural change towards a reduction in car ownership and usage president obamas goals to reduce greenhouse gas emissions align with this trend americans are buying fewer cars and driving less as evidenced by a significant drop in miles driven per person since 2005 this shift predates the economic downturn and continues despite recovery suggesting a deeper cultural change societal factors like internetbased work and socializing urban revitalization and evolving commuting habits are contributing to this trend this reduction in driving has both positive and negative impacts on our society and environment the reduction in driving has positive environmental impacts cars are one of the leading causes of carbon emissions which contribute to global warming by reducing car usage we can reduce carbon emissions and help combat climate change furtherm

/tmp/ipykernel_53064/595753028.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Text: {values[0]}, Target: {values[1]}\n\n")
/tmp/ipykernel_53064/595753028.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Text: {values[0]}, Target: {values[1]}\n\n")
/tmp/ipykernel_53064/595753028.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Text: {values[0]}, Target: {values[1]}\n\n")
/tmp/ipykernel_53064/5957530

In [ ]:
# preprocess for training pipeline
X_train, X_test, y_train, y_test = prep_train_pipeline(df=df_512_token)

### looks good, we should keep in mind that it could lead to overfitting (because copies), so we'll build in regulizations later

In [ ]:
# roberta tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# turn features in input_ids, attention_masks, less training time
train_input_ids, train_attention_masks = tokenize(data=X_train, tokenizer=tokenizer)
test_input_ids, test_attention_masks = tokenize(data=X_test, tokenizer=tokenizer)

In [ ]:
# create the model and show the summary (we dont freeze the backbone, so directly finetune it)

roberta_model = TFRobertaModel.from_pretrained("roberta-base")

model = create_model(roberta_model=roberta_model)
model.summary()

In [ ]:
del df_512_token, X_train, X_test, tokenizer, roberta_model

In [ ]:
# define some callbacks

checkpoint_path = "./512_roberta_model/"
logs = "./logs/"

# save best model (monitor = val_loss)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=0, save_best_only=True
)

# stop training after 6 epochs no improvement, restore best weights
early_stopping = tf.keras.callbacks.EarlyStopping(patience=6, verbose=0, restore_best_weights=True)

# after 3 epochs no improvement, reduce lr (factor = 0.1, so 1e-4 -> 1e-5)
reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(patience=3, min_lr=1e-8)

# tensorboard callback for better visual evaluation
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs)

In [ ]:
callbacks = [model_checkpoint, early_stopping, reduce_lr_on_plateau, tensorboard]

# model ready for training
history = model.fit(
    [train_input_ids, train_attention_masks],
    y_train,
    validation_data=([test_input_ids, test_attention_masks], y_test),
    epochs = 15,
    batch_size = 8,
    callbacks=callbacks
)